In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
#!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
#!pip install folium
import folium

In [2]:
#Ignore ssl certificates
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
url="https://en.wikipedia.org/wiki/Template:Neighbourhoods_of_Pune"
url

'https://en.wikipedia.org/wiki/Template:Neighbourhoods_of_Pune'

In [4]:
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Template:Neighbourhoods of Pune - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XqlZPwpAMMQAAlAnWN0AAACV","wgCSPNonce":!1,"wgCanonicalNamespace":"Template","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":10,"wgPageName":"Template:Neighbourhoods_of_Pune","wgTitle":"Neighbourhoods of Pune","wgCurRevisionId":941587668,"wgRevisionId":941587668,"wgArticleId":26322286,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Neighbourhoods in Pune"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"

In [5]:
neighborhoodList = []
for i in soup.find_all("td",{'class':'navbox-list navbox-odd hlist'})[0].find_all("a"):
    neighborhoodList.append(i.text)
neighborhoodList    

['Aundh',
 'Balewadi',
 'Baner',
 'Bavdhan',
 'Bhosari',
 'Camp',
 'Chakan',
 'Charholi Budruk',
 'Chinchwad',
 'Dapodi',
 'Dehu Road Cantonment',
 'Dehu',
 'Dhayari',
 'Hadapsar',
 'Hinjawadi',
 'Kalewadi',
 'Kalyani Nagar',
 'Khadki cantonment',
 'Kharadi',
 'Koregaon Park',
 'Kothrud',
 'Manjri',
 'Nanded',
 'Pashan',
 'Paud',
 'Pimple Gurav',
 'Pimple Saudagar',
 'Pimpri',
 'Rahatani',
 'Ravet',
 'Sangvi',
 'Sant Tukaram Nagar',
 'Shivajinagar',
 'Sus',
 'Swargate',
 'Thergaon',
 'Viman Nagar',
 'Wakad',
 'Wanowrie',
 'Warje',
 'Yerawada']

In [6]:
neighbourhood = pd.DataFrame({"Neighborhood": neighborhoodList})

neighbourhood.head()

,Neighborhood
0,Aundh
1,Balewadi
2,Baner
3,Bavdhan
4,Bhosari


In [7]:
neighbourhood.shape

(41, 1)

In [8]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Pune, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
coords = [ get_latlng(neighbor) for neighbor in neighbourhood["Neighborhood"].tolist() ]
coords

[[18.563450000000046, 73.81227000000007],
 [18.576020000000028, 73.77983000000006],
 [18.548200000000065, 73.77316000000008],
 [18.517541553892233, 73.77852901740273],
 [18.638730000000066, 73.83748000000008],
 [18.524590000000046, 73.87878000000006],
 [18.734150000000056, 73.85856000000007],
 [18.640720000000044, 73.90397000000007],
 [18.647430000000043, 73.80002000000007],
 [18.494410000000073, 74.39857000000006],
 [18.679160000000024, 73.73255000000006],
 [18.720070000000078, 73.76577000000003],
 [18.447020000000066, 73.80757000000006],
 [18.502530000000036, 73.92706000000004],
 [18.591420000000028, 73.73895000000005],
 [18.394240000000025, 74.05088000000006],
 [18.54645000000005, 73.90067000000005],
 [18.569210000000055, 73.84566000000007],
 [18.544620000000066, 73.93922000000003],
 [18.535330000000044, 73.89382000000006],
 [18.505170000000078, 73.80245000000008],
 [18.481940497085816, 73.86561799417163],
 [18.456420000000037, 73.79200000000003],
 [18.536740000000066, 73.7929000000

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_coords.head()

,Latitude,Longitude
0,18.563450,73.812270
1,18.576020,73.779830
2,18.548200,73.773160
3,18.517542,73.778529
4,18.638730,73.837480


In [11]:
neighbourhood['Latitude']=df_coords['Latitude']
neighbourhood['Longitude']=df_coords['Longitude']
print(neighbourhood.shape)
neighbourhood

(41, 3)


,Neighborhood,Latitude,Longitude
0,Aundh,18.563450,73.812270
1,Balewadi,18.576020,73.779830
2,Baner,18.548200,73.773160
3,Bavdhan,18.517542,73.778529
4,Bhosari,18.638730,73.837480
5,Camp,18.524590,73.878780
6,Chakan,18.734150,73.858560
7,Charholi Budruk,18.640720,73.903970
8,Chinchwad,18.647430,73.800020
9,Dapodi,18.494410,74.398570


In [12]:
address = 'Pune, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune, India 18.521428, 73.8544541.


In [13]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(neighbourhood['Latitude'], neighbourhood['Longitude'], neighbourhood['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [14]:
CLIENT_ID = 'WJLEWTB3NOANHL5A1J4SJ3DTP3OAKPM4PI0CT5SP4A4IHHG1' # your Foursquare ID
CLIENT_SECRET = 'VLJWVMYTEQCTL51ERQM3RTSO4Z0TI4GYOGMIOEDM23I2IZPX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: WJLEWTB3NOANHL5A1J4SJ3DTP3OAKPM4PI0CT5SP4A4IHHG1
CLIENT_SECRET:VLJWVMYTEQCTL51ERQM3RTSO4Z0TI4GYOGMIOEDM23I2IZPX


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    LIMIT=200
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
Pune_venues = getNearbyVenues(names=neighbourhood['Neighborhood'],
                                   latitudes=neighbourhood['Latitude'],
                                   longitudes=neighbourhood['Longitude'])

Aundh
Balewadi
Baner
Bavdhan
Bhosari
Camp
Chakan
Charholi Budruk
Chinchwad
Dapodi
Dehu Road Cantonment
Dehu
Dhayari
Hadapsar
Hinjawadi
Kalewadi
Kalyani Nagar
Khadki cantonment
Kharadi
Koregaon Park
Kothrud
Manjri
Nanded
Pashan
Paud
Pimple Gurav
Pimple Saudagar
Pimpri
Rahatani
Ravet
Sangvi
Sant Tukaram Nagar
Shivajinagar
Sus
Swargate
Thergaon
Viman Nagar
Wakad
Wanowrie
Warje
Yerawada


In [25]:
print(Pune_venues.shape)
Pune_venues.head()

(3383, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aundh,18.56345,73.81227,Crosswords,18.556177,73.809131,Bookstore
1,Aundh,18.56345,73.81227,Mithas,18.554635,73.809369,Dessert Shop
2,Aundh,18.56345,73.81227,Yolkshire,18.553576,73.806888,English Restaurant
3,Aundh,18.56345,73.81227,Pagdandi,18.551972,73.793091,Coffee Shop
4,Aundh,18.56345,73.81227,Lost The Plot,18.554417,73.804808,Lounge


In [22]:
Pune_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aundh,100,100,100,100,100,100
Balewadi,100,100,100,100,100,100
Baner,100,100,100,100,100,100
Bavdhan,100,100,100,100,100,100
Bhosari,42,42,42,42,42,42
Camp,100,100,100,100,100,100
Chakan,5,5,5,5,5,5
Charholi Budruk,9,9,9,9,9,9
Chinchwad,75,75,75,75,75,75


In [27]:
print('There are {} uniques categories.'.format(len(Pune_venues['Venue Category'].unique())))

There are 120 uniques categories.


In [30]:
Pune_venues['Venue Category'].unique()

array(['Bookstore', 'Dessert Shop', 'English Restaurant', 'Coffee Shop',
       'Lounge', 'Ice Cream Shop', 'South Indian Restaurant',
       'Shopping Mall', 'Hotel', 'Asian Restaurant', 'Italian Restaurant',
       'Indian Restaurant', 'Bakery', 'Malay Restaurant', 'BBQ Joint',
       'Multiplex', 'Bistro', 'Breakfast Spot', 'Donut Shop',
       'Chinese Restaurant', 'Café', 'Nightclub', 'Juice Bar',
       'Other Great Outdoors', 'Fast Food Restaurant', 'Brewery', 'Pub',
       'Gym', 'Tea Room', 'Sandwich Place', 'Burger Joint', 'Snack Place',
       'Seafood Restaurant', 'Gym / Fitness Center', 'History Museum',
       'Garden', 'Golf Course', 'Hotel Bar', 'Trail', 'Food Truck', 'Bar',
       'Brazilian Restaurant', 'IT Services', 'French Restaurant',
       'Mexican Restaurant', 'American Restaurant',
       'Maharashtrian Restaurant', 'Electronics Store',
       'Vegetarian / Vegan Restaurant', 'Train Station', 'Pizza Place',
       'Stadium', 'Historic Site', 'Mountain', 'Lake'